In [1]:
import os
import cv2
import pandas as pd
import time
from PIL import Image
import numpy as np
import mediapipe as mp 
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # points will have all 33 landmarks like nose, eyes, ears etc.
mp_pose=mp.solutions.pose


In [2]:
data = list()
data.append('label')
rootdir = "/home/shikha/POSEESTIMATION/TRAIN"


In [3]:
for p in points: #p will iterate through nose, eyes, ears etc...
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data.append("leftElbow")
data.append("rightElbow")
data.append("leftShoulder")
data.append("rightShoulder")
data.append("leftKnee")
data.append("rightKnee")
df = pd.DataFrame(columns = data)

In [4]:
data

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_vis',
 'LEFT_EYE_INNER_x',
 'LEFT_EYE_INNER_y',
 'LEFT_EYE_INNER_z',
 'LEFT_EYE_INNER_vis',
 'LEFT_EYE_x',
 'LEFT_EYE_y',
 'LEFT_EYE_z',
 'LEFT_EYE_vis',
 'LEFT_EYE_OUTER_x',
 'LEFT_EYE_OUTER_y',
 'LEFT_EYE_OUTER_z',
 'LEFT_EYE_OUTER_vis',
 'RIGHT_EYE_INNER_x',
 'RIGHT_EYE_INNER_y',
 'RIGHT_EYE_INNER_z',
 'RIGHT_EYE_INNER_vis',
 'RIGHT_EYE_x',
 'RIGHT_EYE_y',
 'RIGHT_EYE_z',
 'RIGHT_EYE_vis',
 'RIGHT_EYE_OUTER_x',
 'RIGHT_EYE_OUTER_y',
 'RIGHT_EYE_OUTER_z',
 'RIGHT_EYE_OUTER_vis',
 'LEFT_EAR_x',
 'LEFT_EAR_y',
 'LEFT_EAR_z',
 'LEFT_EAR_vis',
 'RIGHT_EAR_x',
 'RIGHT_EAR_y',
 'RIGHT_EAR_z',
 'RIGHT_EAR_vis',
 'MOUTH_LEFT_x',
 'MOUTH_LEFT_y',
 'MOUTH_LEFT_z',
 'MOUTH_LEFT_vis',
 'MOUTH_RIGHT_x',
 'MOUTH_RIGHT_y',
 'MOUTH_RIGHT_z',
 'MOUTH_RIGHT_vis',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_vis',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_vis',
 'LEFT_ELBOW_x',
 'LEFT_

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #ON: c[1] is y coordinate of c and b[1] is the y coordinate of b. that way we calculate the angle in radians
    #ON: then convert radians to angle and then do 360-angle if it is greater than 180 since hand or anything wont move 360 degree.
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
    #if angle<0:
        #angle+=360
        
    return angle 

In [6]:
c=0
num_images=0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            temp=[]    
            one=[]
            imagename=os.path.basename(subdir)
            img = cv2.imread(rootdir + "/" +imagename+"/" +file)
            imageWidth, imageHeight = img.shape[:2]
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)
            one.append(imagename)
            if results.pose_landmarks:
                mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                landmarks = results.pose_landmarks.landmark
                for i,j in zip(points,landmarks):
                    one.append(j.x)
                    one.append(j.y)
                    one.append(j.z)
                    one.append(j.visibility)
            
            leftElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                   [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
            print(leftElbow)
            one.append(leftElbow)
            rightElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
            one.append(rightElbow)


            leftShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
            one.append(leftShoulder)
            rightShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])

            one.append(rightShoulder)
            leftKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
            one.append(leftKnee)
            rightKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
            one.append(rightKnee)
            
            if c==0:
                df=pd.DataFrame([list(one)],columns=data)
                c=c+1
            else:
                df.loc[c]=one
                c=c+1
            num_images += 1
        
        except ValueError as ve:
            continue 
df.to_csv("train_converted.csv", index=False)

105.85073111147973
105.13414828844094
179.51989883317532
145.0215308055094
15.159118697190484
51.95206478967646
6.468142616638129
22.61751468557988
137.19902647179032
137.19902647179032
107.87200393913805
173.89801757100074
38.81252435262986
38.81252435262986
56.03320063142708
48.53909042037264
76.02176245310817
30.535661167622273
121.80669565607619
36.6799070118084
0.11089618804814466
178.20840759201596
67.14533504485789
67.14533504485789
176.21974905174898
174.4257316175658
126.49840145242788
118.2311880164572
77.45549463937063
103.55901931533907
109.00731901211495
78.61088312337247
104.51275605118175
134.34861582936355
52.49444733118714
44.30675928194905
35.62268282731338
175.76529845016483
108.4028223311929
179.63159423289358
117.33985208084226
28.7971366626635
79.01929761080416
79.01929761080416
119.83359747990525
91.66589032613706
79.53264674522035
159.51314267381557
159.51314267381557
106.79786947974063
111.36131456595817
54.72616622969406
178.95393706324447
93.83666683490617
83

160.4847702021152
152.97905681432223
159.7778276948591
159.7778276948591
167.95055080609322
171.53549890482935
167.18207293555807
70.01333710171626
86.55979553625171
170.64400206848242
147.52297247450767
147.52297247450767
167.569363411157
153.64581080774386
134.28852332779505
153.4094656423152
147.1409490304543
165.80267002241624
159.56839385836108
32.693032049909434
166.79849866285153
166.79849866285153
162.56859535826763
165.0998485969105
156.00265112605078
156.00265112605078
171.6231370625114
85.12652435655372
171.50793635134255
126.06713204149793
4.909890070120822
164.8777733792933
171.05568565491757
166.87462863089667
34.23783009555027
168.82723388938066
0.32336382407681535
178.13136923179982
178.13136923179982
161.0540212660373
149.72739796236797
168.20068255558843
176.38642414155697
174.3271692586212
147.70455736372588
26.428856794636886
163.29833046921422
155.9742689481525
162.191729296538
175.7793464048223
169.32972805117933
169.32972805117933
166.55041023689282
179.869840050

34.31998718118542
173.89250328024278
173.89250328024278
174.20393819249927
35.439840137212606
51.974099299727776
51.974099299727776
115.04808112955448
90.68562864322104
168.20321296270438
150.6776735818259
42.621446173491485
166.65310157364993
160.70507683608372
160.70507683608372
173.48842651944838
21.64361077912158
21.64361077912158
59.511642742048906
22.735360085145597
53.37045121830975
132.83289782977135
69.28361413222638
152.5745732607741
23.168645747733926
23.168645747733926
133.11385593265084
176.82272124751572
170.44673840704368
118.84400050761346
166.98829389674637
166.98829389674637
170.66747503945766
148.56625267380613
128.2626225804899
149.30892215850594
174.08705184437784
173.06382101581673
170.85445086330859
156.7889445158583
134.72712762622018
167.96396853418628
165.9230887627868
159.06017218051005
165.95138297411978
146.28036504762224
174.91054350793715
173.67901889032743
175.60929736159645
177.53372830648618
142.43135049736244
178.12991214238554
178.12991214238554
112.

In [7]:
df.head(200)

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_vis,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_vis,LEFT_EYE_x,...,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_vis,leftElbow,rightElbow,leftShoulder,rightShoulder,leftKnee,rightKnee
0,goddess,0.535455,0.136854,-0.417325,1.000000,0.532745,0.114784,-0.358633,1.000000,0.534772,...,0.215182,0.973529,-0.698939,0.998522,105.850731,96.271785,78.997177,71.278112,104.706636,128.817581
1,goddess,0.481509,0.176533,-0.391027,1.000000,0.494633,0.159164,-0.330950,1.000000,0.502304,...,0.194878,0.980698,-0.082552,0.997551,105.134148,105.699485,103.970928,109.808665,119.696478,118.072152
2,goddess,0.490564,0.178078,-0.436347,0.999988,0.499223,0.157603,-0.401645,0.999963,0.505131,...,0.254018,0.907753,-0.337638,0.995580,179.519899,178.218694,22.352400,23.943679,108.062305,108.936671
3,goddess,0.641053,0.306381,-0.578725,0.999952,0.646737,0.290991,-0.538698,0.999906,0.651963,...,0.362300,0.923784,0.030916,0.993731,145.021531,154.469908,137.264701,147.444288,112.854209,147.224011
4,goddess,0.454386,0.365541,-0.221119,0.981740,0.470229,0.350916,-0.207054,0.979907,0.478300,...,0.521656,0.774633,-0.170567,0.952059,15.159119,1.971805,55.746206,48.093558,81.315725,50.597415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,warrior2,0.517484,0.271863,-0.251798,0.994964,0.520407,0.261413,-0.229661,0.994717,0.523206,...,0.498940,0.840324,0.157926,0.787608,176.399536,164.391570,132.246041,41.610303,121.577095,167.338952
196,warrior2,0.574261,0.353241,-0.264670,0.995467,0.567109,0.337566,-0.230472,0.995244,0.566440,...,0.192141,0.861048,-0.284434,0.808614,173.106471,170.359770,96.827038,84.993662,99.522346,175.623624
197,warrior2,0.491928,0.193604,-0.225414,0.995914,0.501167,0.178202,-0.223850,0.995667,0.505491,...,0.254171,0.807118,-0.031543,0.825064,168.658801,178.812523,99.381869,98.325825,179.924182,108.643005
198,warrior2,0.438850,0.285021,-0.141349,0.996320,0.445601,0.269103,-0.153255,0.996089,0.447467,...,0.325866,0.900523,0.027034,0.840939,168.043533,176.353300,100.827345,101.608978,173.974133,111.042018


In [8]:
from sklearn.svm import SVC
da = pd.read_csv("train_converted.csv")


In [9]:
X,Y= df.iloc[:,1:139],df['label']
model = SVC(kernel = 'poly')
model.fit(X,Y)

SVC(kernel='poly')

In [10]:
test= list()
test.append('label')
testdir = "/home/shikha/POSEESTIMATION/TEST"

In [11]:
c=0
test_images=0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            temp=[]    
            one=[]
            imagename=os.path.basename(subdir)
            img = cv2.imread(rootdir + "/" +imagename+"/" +file)
            imageWidth, imageHeight = img.shape[:2]
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)
            one.append(imagename)
            if results.pose_landmarks:
                mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS) #draw landmarks on image
                landmarks = results.pose_landmarks.landmark
                for i,j in zip(points,landmarks):
                    one.append(j.x)
                    one.append(j.y)
                    one.append(j.z)
                    one.append(j.visibility)
            
            leftElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                   [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
            print(leftElbow)
            one.append(leftElbow)
            rightElbow=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
            one.append(rightElbow)


            leftShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
            one.append(leftShoulder)
            rightShoulder=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y])

            one.append(rightShoulder)
            leftKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
            one.append(leftKnee)
            rightKnee=calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                       [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
            one.append(rightKnee)
            
            if c==0:
                test_data=pd.DataFrame([list(one)],columns=data)
                c=c+1
            else:
                test_data.loc[c]=one
                c=c+1
            test_images += 1
        
        except ValueError as ve:
            continue 
test_data.to_csv("test_converted.csv", index=False)

84.40258178854724
102.27549356460025
171.0614714735251
151.06797347243352
145.21823480521272
145.21823480521272
105.67431428479102
106.89226231741122
144.8907780519597
144.8907780519597
107.87200393913805
173.89801757100074
38.81252435262986
38.81252435262986
56.03320063142708
48.53909042037264
76.02176245310817
30.535661167622273
121.80669565607619
36.6799070118084
0.11089618804814466
178.20840759201596
67.14533504485789
67.14533504485789
176.21974905174898
174.4257316175658
126.49840145242788
118.2311880164572
77.45549463937063
103.55901931533907
109.00731901211495
78.61088312337247
104.51275605118175
134.34861582936355
52.49444733118714
44.30675928194905
35.62268282731338
175.76529845016483
108.4028223311929
179.63159423289358
117.33985208084226
28.7971366626635
79.01929761080416
79.01929761080416
119.83359747990525
91.66589032613706
79.53264674522035
159.51314267381557
159.51314267381557
106.79786947974063
111.36131456595817
54.72616622969406
178.95393706324447
93.83666683490617
83

152.97905681432223
159.7778276948591
159.7778276948591
167.95055080609322
171.53549890482935
167.18207293555807
70.01333710171626
86.55979553625171
170.64400206848242
147.52297247450767
147.52297247450767
167.569363411157
153.64581080774386
134.28852332779505
153.4094656423152
147.1409490304543
165.80267002241624
159.56839385836108
32.693032049909434
166.79849866285153
166.79849866285153
162.56859535826763
165.0998485969105
156.00265112605078
156.00265112605078
171.6231370625114
85.12652435655372
171.50793635134255
126.06713204149793
4.909890070120822
164.8777733792933
171.05568565491757
166.87462863089667
34.23783009555027
168.82723388938066
0.32336382407681535
178.13136923179982
178.13136923179982
161.0540212660373
149.72739796236797
168.20068255558843
176.38642414155697
174.3271692586212
147.70455736372588
26.428856794636886
163.29833046921422
155.9742689481525
162.191729296538
175.7793464048223
169.32972805117933
169.32972805117933
166.55041023689282
179.86984005036794
176.35954169

34.31998718118542
173.89250328024278
173.89250328024278
174.20393819249927
35.439840137212606
51.974099299727776
51.974099299727776
115.04808112955448
90.68562864322104
168.20321296270438
150.6776735818259
42.621446173491485
166.65310157364993
160.70507683608372
160.70507683608372
173.48842651944838
21.64361077912158
21.64361077912158
59.511642742048906
22.735360085145597
53.37045121830975
132.83289782977135
69.28361413222638
152.5745732607741
23.168645747733926
23.168645747733926
133.11385593265084
176.82272124751572
170.44673840704368
118.84400050761346
166.98829389674637
166.98829389674637
170.66747503945766
148.56625267380613
128.2626225804899
149.30892215850594
174.08705184437784
173.06382101581673
170.85445086330859
156.7889445158583
134.72712762622018
167.96396853418628
165.9230887627868
159.06017218051005
165.95138297411978
146.28036504762224
174.91054350793715
173.67901889032743
175.60929736159645
177.53372830648618
142.43135049736244
178.12991214238554
178.12991214238554
112.

In [12]:
print(subdir)
print(test_images)

/home/shikha/POSEESTIMATION/TRAIN/plank
1089


In [13]:
test_data.head(20)

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_vis,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_INNER_vis,LEFT_EYE_x,...,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_vis,leftElbow,rightElbow,leftShoulder,rightShoulder,leftKnee,rightKnee
0,goddess,0.559611,0.044733,-0.869396,0.989165,0.548590,0.030528,-0.868077,0.989334,0.550694,...,0.688710,0.926312,0.202465,0.783026,84.402582,148.099861,125.634194,104.342612,148.470191,129.772560
1,goddess,0.521078,0.159093,-1.398114,0.990245,0.531839,0.135620,-1.354568,0.990399,0.538049,...,0.517391,1.107735,-0.759699,0.794502,102.275494,92.885681,99.399149,102.631558,179.949198,176.802202
2,goddess,0.483392,0.202281,-0.903376,0.991211,0.492092,0.191318,-0.879712,0.991346,0.498631,...,0.326246,0.776421,-0.284530,0.804132,171.061471,176.335629,36.107510,59.339071,144.566739,99.728046
3,goddess,0.591800,0.271648,-0.725649,0.992059,0.597632,0.252653,-0.711564,0.992182,0.602047,...,0.586409,0.955777,0.372084,0.821585,151.067973,152.461968,157.864619,176.656199,171.071469,156.516925
4,goddess,0.493168,0.336246,-0.088961,0.988294,0.490647,0.318583,-0.058381,0.989635,0.493161,...,0.381072,0.807767,-0.252116,0.796039,145.218235,61.260871,74.145053,82.597777,110.099062,74.876613
5,goddess,0.481337,0.300642,-0.221392,0.989464,0.486761,0.284903,-0.194350,0.990671,0.490520,...,0.281590,0.929919,-0.079726,0.815123,105.674314,107.892620,106.541882,101.574994,125.980861,126.093613
6,goddess,0.488642,0.409856,-0.376019,0.990517,0.493012,0.394426,-0.351703,0.991602,0.496973,...,0.319562,0.929052,-0.116049,0.832275,106.892262,116.163813,104.509044,109.965018,101.539211,105.336486
7,goddess,0.550186,0.280025,-0.144898,0.991446,0.567010,0.259776,-0.191221,0.992425,0.574399,...,0.717870,0.893964,0.673387,0.793257,144.890778,152.795415,9.522460,7.631933,16.527125,27.767982
8,goddess,0.531864,0.231331,-0.228046,0.992300,0.521170,0.215858,-0.293720,0.993182,0.509493,...,0.853058,0.896874,1.163726,0.794824,107.872004,116.743718,115.035716,120.551558,109.416646,126.343179
9,goddess,0.500958,0.185950,-0.390285,0.991992,0.506163,0.160441,-0.371786,0.992334,0.511637,...,0.252834,0.890651,-0.023494,0.800791,173.898018,170.991182,4.237580,13.146390,111.791787,114.378510


In [14]:
S,T= test_data.iloc[:,1:139],test_data['label']
pred = model.predict(S)


In [15]:
print(pred)

['goddess' 'downdog' 'cobra' ... 'plank' 'plank' 'plank']


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(T,pred)

0.6235078053259872